In [1]:
# self-define function
from data_cleaning import cleaning_text, sentence_df
from participant_function import participants_list, clean_participants_list
from stock_price_info import stock_price_change

# read and saving path
# path = "/Users/timliu/Documents/GitHub/final_01/Transcript_test"
# path = "/Users/timliu/Documents/GitHub/final_01/Tim_test"
# save_path = "/Users/timliu/Documents/GitHub/final_01/Output"

path = "/Users/hienanh/Documents/GitHub/final_01/Transcript_test"
save_path = "/Users/hienanh/Documents/GitHub/final_01/Output"

# other packages
import pdftotext
import pandas as pd
import numpy as np
import re
import os
import string
# from collections import Counter
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
# from sklearn.linear_model import LogisticRegression

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

import warnings
from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# all files path
company_paths = []
sectors = os.listdir(path) 
if '.DS_Store' in sectors:
    sectors.remove('.DS_Store')

for sector in sectors:
    # path to each sector files
    sector_path = path+"/"+sector
    
    # path to each company files
    companies = os.listdir(sector_path)
    if '.DS_Store' in companies:
        companies.remove('.DS_Store')
    
    for company in companies:
        tmp = sector_path+"/"+company
        company_paths.append(tmp)

In [3]:
# # checking with DF (not necessary)
# company_paticipants_df = pd.DataFrame()
# other_paticipants_df = pd.DataFrame()

df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index']) # create a dataframe with 2500 rows
# all_participants = []      
company_paticipants_list = [] 
other_paticipants_list = []

for single_path in company_paths:
    files = os.listdir(single_path)
    for file in files:
        # print(file)
        if file.endswith(".pdf"):
            # print(file)
            # Load PDF
            with open(single_path+"/"+file, "rb") as f:
                pdf = pdftotext.PDF(f)
            # Save all text to a txt file.
            with open(save_path+"/"+file.replace(".pdf", ".txt"), "w") as f:
                f.write("\n\n".join(pdf))
            # open the text file
            with open(save_path+"/"+file.replace(".pdf", ".txt")) as f:
                contents = f.readlines()
                df_clean = cleaning_text(contents)    ##### here we use the function from data_cleaning.py
                
                index_presentation = df_clean[df_clean[0] == 'Presentation'].index
                if len(index_presentation) == 2:
                    df_clean.loc[index_presentation[0], 0] = 'heading' # replace the first index_presentation in df_clean with "head" # interacting with the participant_function.py

                # extract the participants
                tmp_df_text,tmp_company_paticipants,tmp_other_paticipants = participants_list(df_clean) ##### here we use the function from participant_function.py
                company_paticipants_list.append(tmp_company_paticipants)
                other_paticipants_list.append(tmp_other_paticipants)
                

                # # checking with DF (not necessary)
                # company_paticipants = pd.DataFrame(company_paticipants)
                # if company_paticipants.empty:
                #     company_paticipants = pd.DataFrame(np.zeros((1,1)), columns=['index'])
                # company_paticipants_df[f"{files[files.index(file)]}"] = company_paticipants
                # other_paticipants = pd.DataFrame(other_paticipants)
                # if other_paticipants.empty:
                #     other_paticipants = pd.DataFrame(np.zeros((1,1)), columns=['index'])
                # other_paticipants_df[f"{files[files.index(file)]}"] = other_paticipants
                
                df = pd.DataFrame()
                #df[f"{files[files.index(file)]}"] = tmp_df_text
                #df[file] = tmp_df_text
                # remove the row using re, if the row ends with Presentation  Investor Day	
                #df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)
                df_clean_na[file] = tmp_df_text.dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na.head(5)

,20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf,20141119_Sompo_Holdings_Inc-_Earnings_Call_2014-11-24_FS000000002417374854.pdf,20151118_Sompo_Holdings_Inc-_Earnings_Call_2015-11-20_DN000000002241138647.pdf,20131119_Sompo_Holdings_Inc-_Earnings_Call_2013-11-25_DN000000002116348219.pdf,20181119_Sompo_Holdings_Inc-_Earnings_Call_2018-11-19_SD000000002905610039.pdf,20151126_Sompo_Holdings_Inc-_Conf-Presentation_Call_2015-11-26_SD000000002889371132.pdf,...,20210810_Munich_Re-_Earnings_Call_2021-8-10_DN000000002962247258.pdf,20201208_Munich_Re-_Shareholder_Mtg_Call_2020-12-8_DN000000002944723654.pdf,20200507_Munich_Re-_Earnings_Call_2020-5-7_RT000000002834670399.pdf,20171109_Munich_Re-_Earnings_Call_2017-11-9_FS000000002383615110.pdf,20170509_Munich_Re-_Earnings_Call_2017-5-9_FS000000002450443745.pdf,20170315_Munich_Re-_Earnings_Call_2017-3-15_FS000000002336461963.pdf,20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf
0,Shinichi Hara,Kengo Sakurada,Unverified Participant,Osamu Nose,Shinichi Hara,Yukinori Kuroda,Yukinori Kuroda,Shinichi Hara,Yukinori Kuroda,Kengo Sakurada,...,Operator,"Good afternoon, ladies and gentlemen and warm ...",Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Christian Becker-Hussong,Joerg Schneider
1,I am Hara from NKSJ Holdings. Thank you very m...,My name is Sakurada from Sompo Holdings. Thank...,I thank all of you for gathering here today fo...,"Hello. I am Osamu Nose, Head of Investor Relat...",Good evening. This is Shinichi Hara of NKSJ Ho...,"Hello, everyone. I'm Kuroda of Investor Relati...",So this is Kuroda from Sompo Japan Nipponkoa H...,I'm Hara from NK Share Holdings. Thank you ver...,I'm Kuroda with Sompo Holdings. Thank you very...,"As introduced, I am Sakurada, the Group CEO. F...",...,"Good day, and welcome to the Munich Re Half Ye...",very -- we very much hope you are all well and...,"Good day, and welcome to the Muenchener Quarte...",Thanks for joining us for our Q3 earnings call...,"Good day, ladies and gentlemen, and welcome to...","Ladies and gentlemen, welcome to the Munich Re...","Good day, ladies and gentlemen. Welcome to the...",transformation. Welcome to everybody here in t...,Thank you. Ladies and gentlemen a warm welcome...,"Ladies and gentlemen, welcome to the presentat..."
2,sure you're tired by now at this late hour but...,expense of your very busy schedule.,are as we have announced on the 18th. Therefor...,Holdings. Thank you for participating in our c...,busy schedule to attend this conference call. ...,Thank you very much for joining us today in th...,participating in today's conference call.,"call for the first half results. Today, I'll c...",schedule to this conference call. I would like...,meeting despite your busy schedule. It seems t...,...,Conference Call. Today's conference is being r...,"Before we start, please allow us a few houseke...",Results Conference Call. Today's conference is...,"very straightforward, after my short introduct...",Q1 Conference Call. Today's conference is bein...,room. And of course also a very warm welcome t...,Statement Conference Call. Today's conference ...,everybody following this conference on the Int...,Financial Results 2018 and Strategic Outlook. ...,Schneider and I am the CFO of Munich Re Group.
3,What I am going to explain today is only the f...,We just made the announcement of the results o...,"for myself toda

In [4]:
# generating all the participants
company_paticipants_list = clean_participants_list(company_paticipants_list)
other_paticipants_list = clean_participants_list(other_paticipants_list)
all_participants_list_tmp = []
all_participants_list_tmp = company_paticipants_list + other_paticipants_list

# drop the duplicates
all_participants_list = []
for x in all_participants_list_tmp:
    if x not in all_participants_list:
        all_participants_list.append(x)


In [40]:
concat_df = pd.DataFrame()
participants_index =pd.DataFrame(np.zeros((2500,1)), columns=['index'])
for column in df_clean_na.columns:
    # identify all the rows in df with all_participants_list in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants_list)].index.tolist()
    # apply the both_participants_row_index to the df_clean_na['participants']
    concat_df[column] = df_clean_na[column]
    ppt_list = df_clean_na[column].apply(lambda x: x if x in all_participants_list else 0)
    idx_list = [(ppt_list[idx]+str(idx)) if ppt_list[idx] != 0 else np.nan for idx in range(len(ppt_list)) ]
    # adding paricipants name
    concat_df[f"participants_{column}"] = [name if name != 0 else np.nan for name in ppt_list]
    # adding index columns
    concat_df[f"idx_{column}"] = idx_list
    # fill the NaN with the value of the previous row
    concat_df[f"participants_{column}"] = concat_df[f"participants_{column}"].fillna(method='ffill')
    concat_df[f"idx_{column}"] = concat_df[f"idx_{column}"].fillna(method='ffill')
    # # create df with the index of the row with all_participants_list # got some issue here
    # participants_index[f"participants_index_{column}"] = concat_df[f"participants_{column}"].apply(lambda x: concat_df[column].index[concat_df[column] == x].tolist()[0] if x in all_participants_list else np.nan)

concat_df


,20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,participants_20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,idx_20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,participants_20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,idx_20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,participants_20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,idx_20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,...,idx_20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,participants_20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,idx_20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,participants_20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,idx_20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf,participants_20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf,idx_20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf
0,Shinichi Hara,Shinichi Hara,Shinichi Hara0,Kengo Sakurada,Kengo Sakurada,Kengo Sakurada0,Unverified Participant,Unverified Participant,Unverified Participant0,Osamu Nose,...,Operator0,Christian Becker-Hussong,Christian Becker-Hussong,Christian Becker-Hussong0,Christian Becker-Hussong,Christian Becker-Hussong,Christian Becker-Hussong0,Joerg Schneider,Joerg Schneider,Joerg Schneider0
1,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,Shinichi Hara0,My name is Sakurada from Sompo Holdings. Thank...,Kengo Sakurada,Kengo Sakurada0,I thank all of you for gathering here today fo...,Unverified Participant,Unverified Participant0,"Hello. I am Osamu Nose, Head of Investor Relat...",...,Operator0,transformation. Welcome to everybody here in t...,Christian Becker-Hussong,Christian Becker-Hussong0,Thank you. Ladies and gentlemen a warm welcome...,Christian Becker-Hussong,Christian Becker-Hussong0,"Ladies and gentlemen, welcome to the presentat...",Joerg Schneider,Joerg Schneider0
2,sure you're tired by now at this late hour but...,Shinichi Hara,Shinichi Hara0,expense of your very busy schedule.,Kengo Sakurada,Kengo Sakurada0,are as we have announced on the 18th. Therefor...,Unverified Participant,Unverified Participant0,Holdings. Thank you for participating in our c...,...,Operator0,everybody following this conference on the Int...,Christian Becker-Hussong,Christian Becker-Hussong0,Financial Results 2018 and Strategic Outlook. ...,Christian Becker-Hussong,Christian Becker-Hussong0,Schneider and I am the CFO of Munich Re Group.,Joerg Schneider,Joerg Schneider0
3,What I am going to explain today is only the f...,Shinichi Hara,Shinichi Hara0,We just made the announcement of the results o...,Kengo Sakurada,Kengo Sakurada0,"for myself today, I will be mainly talking abo...",Unverified Participant,Unverified Participant0,I would like to go through our business result...,...,Operator0,We are delighted to have 11 colleagues with us...,Christian Becker-Hussong,Christian Becker-Hussong0,we get started I would like to express some ou...,Christian Becker-Hussong,Christian Becker-Hussong0,"As usual, I will provide you with the main mes...",Joerg Schneider,Joerg Schneider0
4,full fiscal year of 2012 and shareholder retur...,Shinichi Hara,Shinichi Hara0,like to present the progress of the midterm pl...,Kengo Sakurada,Kengo Sakurada0,group.,Unverifie

In [53]:
# use len(concat_df.columns.to_list()) to write a for loop
model_df = pd.DataFrame()
for i in range(int(len(concat_df.columns.to_list())/3)):
    #print(i)
    tmp_df = pd.DataFrame()
    tmp_df = concat_df.iloc[:,(i*3):(i*3)+3].copy()
    # extract the index as column from the text
    tmp_df['file_name'] = tmp_df.columns.to_list()[0]
    # extract the date from the index column
    tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
    # change the date column to datetime
    tmp_df['date'] = pd.to_datetime(tmp_df['date'])
    # rename to be consistent with the column name
    tmp_df.columns = ["line", "participants",  "idx", "file_name","date"]
    # if the 'participants' column's value equals to any of the company_paticipants_list, other_paticipants_list, then set the value to 0
    tmp_df['company_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in company_paticipants_list else 0)
    tmp_df['other_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in other_paticipants_list else 0)
    # drop the row if the column "line" is NaN
    tmp_df = tmp_df.dropna(subset=['line'], how='all')
    tmp_df['company_name1']  = tmp_df['file_name'].apply(lambda x: x.split('_')[1])
    tmp_df['company_name2']  = tmp_df['file_name'].apply(lambda x: x.split('_')[2])
    tmp_df['company_name'] = tmp_df["company_name1"] + " " + tmp_df["company_name2"]
    # drop the 'company_name1' and 'company_name2' column
    tmp_df = tmp_df.drop(columns=['company_name1', 'company_name2']).reset_index(drop=True)
    # drop line contains only participants name
    id_rows = tmp_df[tmp_df['line']==tmp_df['participants']].index
    tmp_df = tmp_df.drop(id_rows)
    # append into dataframe
    model_df = model_df.append(tmp_df)

model_df

,line,participants,idx,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name
1,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,Shinichi Hara0,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0,Sompo Holdings
2,sure you're tired by now at this late hour but...,Shinichi Hara,Shinichi Hara0,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0,Sompo Holdings
3,What I am going to explain today is only the f...,Shinichi Hara,Shinichi Hara0,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0,Sompo Holdings
4,full fiscal year of 2012 and shareholder retur...,Shinichi Hara,Shinichi Hara0,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0,Sompo Holdings
5,"NKSJ Group management plan announced today, Pr...",Shinichi Hara,Shinichi Hara0,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0,Sompo Holdings
...,...,...,...,...,...,...,...,...
146,"profits should be clearly lower. However, in t...",Joerg Schneider,Joerg Schneider0,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,1,0,Munich Re-
147,"positive bottom line impacts from goodwill, ta...",Joerg Schneider,Joerg Schneider0,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,1,0,Munich Re-
148,course of the annual closing. I do not see why...,Joerg Schneider,Joerg Schneider0,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,1,0,Munich Re-
149,Thank you very much for listening. Nikolaus vo...,Joerg Schneider,Joerg Schneider0,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,1,0,Munich Re-


In [54]:
# model_df groupby 'participants','file_name','date','company_paticipants_yes','other_paticipants_yes','company_name' with ['sentence'] and .apply(' '.join)
model_df = model_df.groupby(['participants','idx','file_name','date','company_paticipants_yes','other_paticipants_yes','company_name'])['line'].apply(' '.join).reset_index()
# model_df rename the column 'sentence' to 'paraghrph'
model_df.columns = ['participants','idx','file_name','date','company_paticipants_yes','other_paticipants_yes','company_name','paragraph']

In [55]:
model_df

,participants,idx,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name,paragraph
0,Adrian Jones,Adrian Jones1868,20180905_SCOR_SE-_Shareholder_Mtg_Call_2018-9-...,2018-09-05,1,0,SCOR SE-,"Thanks, Ian. Wanted to talk a little bit about..."
1,Akira Harashima,Akira Harashima1,20191003_TOKIO_MARINE_HD-_M-A_Call_2019-10-3_S...,2019-10-03,1,0,TOKIO MARINE,My name is Harashima from Tokio Marine Holding...
2,Akira Harashima,Akira Harashima348,20191125_TOKIO_MARINE_HD-_Conf-Presentation_Ca...,2019-11-25,1,0,TOKIO MARINE,This is Harashima speaking. Let me just add a ...
3,Akira Harashima,Akira Harashima435,20210527_TOKIO_MARINE_HD-_Shareholder_Mtg_Call...,2021-05-27,1,0,TOKIO MARINE,"This is Harashima, and thank you very much for..."
4,Akira Harashima,Akira Harashima468,20191125_TOKIO_MARINE_HD-_Conf-Presentation_Ca...,2019-11-25,1,0,TOKIO MARINE,"Yes, I would like to follow up on his comments..."
...,...,...,...,...,...,...,...,...
18170,Yusuke Yabumoto,Yusuke Yabumoto414,20150807_TOKIO_MARINE_HD-_Earnings_Call_2015-8...,2015-08-07,0,1,TOKIO MARINE,Thank you. I would like to ask you two questio...
18171,Yusuke Yabumoto,Yusuke Yabumoto432,20150807_TOKIO_MARINE_HD-_Earnings_Call_2015-8...,2015-08-07,0,1,TOKIO MARINE,Okay. Thank you.
18172,Yusuke Yabumoto,Yusuke Yabumoto515,20151125_MS-AD_INSURANCE-_Earnings_Call_2015-1...,2015-11-25,0,1,MS-AD INSURANCE-,"Yabumoto, Mizuho Securities. Let me ask two qu..."
18173,Yuta Tachizako,Yuta Tachizako110,20211125_Sompo_Holdings_Inc-_M-A_Call_2021-11-...,2021-11-25,1,0,Sompo Holdings,(through translator) As a Nursing Care Facilit...


In [56]:
model_df['paragraph'][0]

"Thanks, Ian. Wanted to talk a little bit about what we're doing in Ventures and put that into the broader context of how we think about new business development in the P&C division. Because I was talking about Ventures, I had intended to come today dressed in the typical Silicon Valley Ventures sort of outfit. And so I wear that to the office one day and Denis said, why are you wearing pajamas? So that was the end of that. Anyway, what we're working on, on the P&C side, I want to put it in the broader context. You've heard a little bit about some of the internal development that we've done. When there is a system or a process that is going to drive our competitive advantage, we develop that internally. And so all of our core systems have been developed internally off of the Omega backbone. At the far side of the slide then, there are a number of market-wide initiatives, which have also been discussed today. Where there is a big challenge and where it makes sense to do it in partnershi

In [57]:
model_df['paragraph'][109]

'[Question Inaudible].'

In [58]:
model_df['paragraph'][110]

'[Question Inaudible]'

In [59]:
model_df['paragraph'][108]

'Thank you. [Question Inaudible].'

In [9]:
# # using re, if test['participants'] shows up in the test['paraghrph'] then replace it with ''
# for i in range(len(model_df)):
#     model_df['paraghrph'][i] = model_df['paraghrph'][i].apply(lambda x: re.sub(r'\b'+model_df['participants'].to_list()[i]+r'\b', '', x))

In [33]:
# dataframe company and its tickers 這邊我們只取兩個categories的資料
stock_list = [
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8766.T','8766.T', # this company has 3 file name
    '8630.T','8725.T','QBE.AX',
]
company_name = [
    # European (Re)Insurers
    'SCOR SE-','Munich Re-','Swiss Re','Hannover Rueck',
    # Japanese & Pacific
    'TOKIO MARINE', 'HCC INSURANCE-','MITSUI FUDOSAN-',
    'Sompo Holdings','MS-AD INSURANCE-','QBE INSURANCE-'
               ]
company_ticker_df = pd.DataFrame({'company_name':company_name,
                                  'ticker':stock_list})

company_ticker_df

,company_name,ticker
0,SCOR SE-,SCR.PA
1,Munich Re-,MUV2.DE
2,Swiss Re,SREN.SW
3,Hannover Rueck,HNR1.DE
4,TOKIO MARINE,8766.T
5,HCC INSURANCE-,8766.T
6,MITSUI FUDOSAN-,8766.T
7,Sompo Holdings,8630.T
8,MS-AD INSURANCE-,8725.T
9,QBE INSURANCE-,QBE.AX


In [39]:
stock_price_df = stock_price_change(stock_list, '2010-12-01', '2022-01-30')  ##### here we use the function from stock_price_change.py

# merged company tickers into big dataframe
prc_n_tkr_df = model_df.copy()
prc_n_tkr_df = prc_n_tkr_df.join(company_ticker_df.set_index('company_name'), 
                         on='company_name',
                         how='left'
                        )
#print(len(prc_n_tkr_df))
# merged stock price in
#merged_data= prc_n_tkr_df.merge(stock_price_df, on=["date","ticker"], how='left')
merged_data = prc_n_tkr_df.join(stock_price_df.set_index(["date","ticker"]), 
                         on=["date","ticker"],
                         how='left'
                        )
# drop na rows, which date in on Sat or Sun so it did not have the stock price
# merged_data.loc[merged_data['Close'].isnull()==True]
#merged_data = merged_data.dropna()
merged_data = merged_data.reset_index(drop=True)
merged_data

,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name,paragraph,ticker,Close,D-1,...,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Adrian Jones,20180905_SCOR_SE-_Shareholder_Mtg_Call_2018-9-...,2018-09-05,1,0,SCOR SE-,"Adrian Jones Thanks, Ian. Wanted to talk a lit...",SCR.PA,37.750000,-0.027814,...,0.064880,-0.014570,0.081352,-0.009271,0.073663,0.001060,0.052412,0.034437,0.116534,0.067815
1,Akira Harashima,20191003_TOKIO_MARINE_HD-_M-A_Call_2019-10-3_S...,2019-10-03,1,0,TOKIO MARINE,Akira Harashima My name is Harashima from Toki...,8766.T,5718.000000,-0.017863,...,-0.011411,0.001574,-0.010555,-0.007870,-0.049060,-0.029556,-0.018032,-0.003148,-0.002964,-0.023085
2,Akira Harashima,20191125_TOKIO_MARINE_HD-_Conf-Presentation_Ca...,2019-11-25,1,0,TOKIO MARINE,Akira Harashima This is Harashima speaking. Le...,8766.T,5943.000000,0.002192,...,0.004904,-0.001346,0.001517,0.003534,0.010027,0.012956,-0.000505,0.040384,0.018858,0.048629
3,Akira Harashima,20210527_TOKIO_MARINE_HD-_Shareholder_Mtg_Call...,2021-05-27,1,0,TOKIO MARINE,"Akira Harashima This is Harashima, and thank y...",8766.T,5130.000000,-0.015544,...,-0.024715,0.009552,-0.028041,0.017739,-0.031893,0.024951,-0.044337,0.000780,-0.031344,0.005653
4,Alexa Winnik,20191113_Swiss_Re_AG-_Guidance_Call_2019-11-13...,2019-11-13,1,0,Swiss Re,I think we can begin. So everyone's ready. All...,SREN.SW,105.800003,-0.005172,...,-0.005172,0.002363,-0.007970,0.003781,0.000946,0.006144,0.015843,0.009924,0.028183,0.007089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3349,Yusuke Yabumoto,20150306_Sompo_Holdings_Inc-_Guidance_Call_201...,2015-03-06,0,1,Sompo Holdings,Yusuke Yabumoto I have two questions. The firs...,8630.T,3771.000000,0.019189,...,0.014801,0.011005,-0.001456,0.001591,0.013165,0.048263,0.068272,0.024927,0.108140,0.005436
3350,Yusuke Yabumoto,20150807_TOKIO_MARINE_HD-_Earnings_Call_2015-8...,2015-08-07,0,1,TOKIO MARINE,Yusuke Yabumoto Thank you. I would like to ask...,8766.T,5318.000000,0.020142,...,0.039484,-0.004513,0.038064,0.003385,0.029622,0.011659,0.035033,-0.097029,0.025058,-0.079353
3351,Yusuke Yabumoto,20151118_Sompo_Holdings_Inc-_Earnings_Call_201...,2015-11-18,0,1,Sompo Holdings,"Yusuke Yabumoto Thank you, very much. My first...",8630.T,3952.000000,-0.003530,...,0.001521,0.027075,-0.011753,0.005820,-0.027320,-0.000759,0.034555,-0.008856,0.020134,-0.031630
3352,Yusuke Yabumoto,20151125_MS-AD_INSURANCE-_Earnings_Call_2015-1...,2015-11-25,0,1,MS-AD INSURANCE-,"Yusuke Yabumoto Yabumoto, Mizuho Securities. L...",8725.T,3604.000000,-0.016912,...,-0.023571,-0.006659,-0.032743,-0.036626,-0.048072,-0.013319,-0.052078,-0.032186,0.025612,-0.064650


In [36]:
merged_data['paragraph'][0]

"Adrian Jones Thanks, Ian. Wanted to talk a little bit about what we're doing in Ventures and put that into the broader context of how we think about new business development in the P&C division. Because I was talking about Ventures, I had intended to come today dressed in the typical Silicon Valley Ventures sort of outfit. And so I wear that to the office one day and Denis said, why are you wearing pajamas? So that was the end of that. Anyway, what we're working on, on the P&C side, I want to put it in the broader context. You've heard a little bit about some of the internal development that we've done. When there is a system or a process that is going to drive our competitive advantage, we develop that internally. And so all of our core systems have been developed internally off of the Omega backbone. At the far side of the slide then, there are a number of market-wide initiatives, which have also been discussed today. Where there is a big challenge and where it makes sense to do it 